In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from tqdm import tqdm

In [2]:
pkn_kerken  = requests.get('https://protestantsekerk.nl/kerkzoeker/?json').json()['churches']

In [6]:
def extract_emails_from_page(url: str) -> list[str]:
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")
        emails = re.findall(
            r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", soup.text
        )
    except Exception as e:
        emails = [e]
    return emails


kerk_urls = [kerk["website"] for kerk in pkn_kerken[:50]]
kerk_emails = {}

for kerk_url in tqdm(kerk_urls):
    emails = extract_emails_from_page(kerk_url)
    kerk_emails[kerk_url] = emails

100%|██████████| 50/50 [00:39<00:00,  1.27it/s]


In [7]:
# Prepare data for DataFrame
rows = []
for key, values in kerk_emails.items():
    if values:
        for value in values:
            rows.append((key, value))
    else:
        rows.append((key, None))

# Create DataFrame
df = pd.DataFrame(rows, columns=['URL', 'Email'])
df = df[df['Email'].str.contains(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', na=False)]

In [8]:
df.to_excel('kerk_emails_via_hoofdpagina.xlsx')


In [29]:
kerk_emails